#  Working with SQLite in Python: SELECT, INSERT , UPDATE



In [114]:
import sqlite3

##  Connect to SQLite

In [115]:
db_name = "INSTRUCTOR.db"
conn = sqlite3.connect(db_name)

## Create cursor

In [116]:
cursor_obj = conn.cursor()

## The Instructor table is dropped if it exists before creation

In [117]:
cursor_obj.execute("DROP TABLE IF EXISTS INSTRUCTOR")

## Create Table 

In [118]:
table_sql = """
CREATE TABLE IF NOT EXISTS INSTRUCTOR (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    FNAME VARCHAR(20),
    LNAME VARCHAR(20),
    CITY VARCHAR(20),
    CCODE CHAR(2)
);
"""

cursor_obj.execute(table_sql)
print("Table is Ready")


Table is Ready


## Insert (1 row)

In [119]:
insert_one_sql = """
INSERT INTO INSTRUCTOR (fname, lname , city , ccode) 
VALUES ('Rav', 'Ahuja', 'TORONTO', 'CA')
"""

In [120]:
cursor_obj.execute(insert_one_sql)

In [121]:
conn.commit()

## Insert (multiple rows in a single statement)

In [122]:
insert_many_sql = """
INSERT INTO INSTRUCTOR (fname, lname, city, ccode)
VALUES
    ('Raul', 'Chong', 'Markham', 'CA'),
    ('Hima', 'Vasudevan', 'Chicago', 'US'),
    ('Antonio', 'Rivera', 'Texas', 'TX');
"""


In [123]:
cursor_obj.execute(insert_many_sql)

In [124]:
conn.commit()

## SELECT + fetchall()

In [125]:
statement = "SELECT * FROM INSTRUCTOR"

In [126]:
cursor_obj.execute(statement)

In [127]:
results_all = cursor_obj.fetchall()

In [128]:
for row in results_all:
     print(row)

(1, 'Rav', 'Ahuja', 'TORONTO', 'CA')
(2, 'Raul', 'Chong', 'Markham', 'CA')
(3, 'Hima', 'Vasudevan', 'Chicago', 'US')
(4, 'Antonio', 'Rivera', 'Texas', 'TX')


## SELECT + fetchmany(2)

In [129]:
cursor_obj.execute(statement)

In [130]:
results_two = cursor_obj.fetchmany(2)

In [131]:
for row in results_two:
    print(row)

(1, 'Rav', 'Ahuja', 'TORONTO', 'CA')
(2, 'Raul', 'Chong', 'Markham', 'CA')


## UPDATE (change Rav’s city)

In [132]:
update_sql = """
UPDATE INSTRUCTOR SET city = 'MOOSETOWN' WHERE id = 1
"""  

In [133]:
cursor_obj.execute(update_sql)

In [134]:
conn.commit()

In [135]:
cursor_obj.execute(statement)

In [136]:
results_all = cursor_obj.fetchall()

In [137]:
for row in results_all:
    print(row)

(1, 'Rav', 'Ahuja', 'MOOSETOWN', 'CA')
(2, 'Raul', 'Chong', 'Markham', 'CA')
(3, 'Hima', 'Vasudevan', 'Chicago', 'US')
(4, 'Antonio', 'Rivera', 'Texas', 'TX')


## Convert to pandas using read_sql_query

In [138]:
import pandas as pd 

In [139]:
df = pd.read_sql_query("SELECT * FROM INSTRUCTOR ;" , conn)

In [140]:
df

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US
3,4,Antonio,Rivera,Texas,TX


## Mini operations

### Select

In [141]:
df = pd.read_sql_query("SELECT FNAME , CITY FROM INSTRUCTOR " , conn)
df

,FNAME,CITY
0,Rav,MOOSETOWN
1,Raul,Markham
2,Hima,Chicago
3,Antonio,Texas


In [142]:
df = pd.read_sql_query("""
SELECT * 
FROM INSTRUCTOR
WHERE CCODE = 'CA' """ , conn)
df

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA


### Update

In [143]:
update_sql = """ 
UPDATE INSTRUCTOR
SET CITY = 'Los Angeles'
WHERE city = 'MOOSETOWN' OR city = 'Markham'

"""

In [144]:
cursor_obj.execute(update_sql)

In [ ]:
conn.commit()

In [147]:
df = pd.read_sql_query("""
SELECT * 
FROM INSTRUCTOR
WHERE city = 'Los Angeles' """ ,conn)
df

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,Los Angeles,CA
1,2,Raul,Chong,Los Angeles,CA


### Delete

In [150]:
delete_sql = "DELETE FROM INSTRUCTOR WHERE ID = '3' "

In [151]:
cursor_obj.execute(delete_sql)

In [152]:
conn.commit()

In [153]:
df = pd.read_sql_query("SELECT * FROM INSTRUCTOR", conn)
df

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,Los Angeles,CA
1,2,Raul,Chong,Los Angeles,CA
2,4,Antonio,Rivera,Texas,TX


## Additional pandas

### Create a new column by merging two columns

In [156]:
df["FULLNAME"] = df["FNAME"] + " " + df["LNAME"]


### Keep only the instructors from Los Angeles

In [157]:
df_filtered = df[df["CITY"] == "Los Angeles"]


### Export the result to a .csv file

In [158]:
df_filtered.to_csv("instructors.csv", index=False)


In [159]:
df = pd.read_csv("instructors.csv")

In [160]:
df

,ID,FNAME,LNAME,CITY,CCODE,FULLNAME
0,1,Rav,Ahuja,Los Angeles,CA,Rav Ahuja
1,2,Raul,Chong,Los Angeles,CA,Raul Chong


## Close connection

In [ ]:
conn.close()